In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import Double_Null_utils as dnu
from scipy import optimize
import tables
from mpmath import *


In [2]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
vscalarfield=False


In [3]:
Tol=1
N=25
Nul=1000
Nup=10**(40)
scal=1
umax=5.3
vmax=200
datatype=np.float64


#ru0=5.0
#dr0v=.4
ru0=2.0
#dr0v=0.045
dr0v=-1/(4*-.375)*(1-2*M0/ru0+Q**2/ru0**2)
#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 1.31224989991992
r-: 0.6877501000800801
rc: 0.0
Number of points for highest iteration is 2650000000.0,530000.0X5000
Total number of points is 2650000000
Runtime: about 566.24 minutes


In [5]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nv),dtype=datatype)
signpf=np.zeros((2,Nv),dtype=datatype)
phinpf=np.zeros((2,Nv),dtype=datatype)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
dsigunpv=np.zeros((1,Nv),dtype=datatype)


rnpf[0], signpf[0], phinpf[0] = dnu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,datatype)




Using Standard Coordinates Along V


In [6]:
drunptemp=np.empty((Nv))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.array([0.0],dtype=datatype)
drunp=np.array([0.0],dtype=datatype)
dsigunp=np.array([0.0],dtype=datatype)
dphinpuf=np.array([0.0],dtype=datatype)
massnpf[0][0]=1.0

drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda*ru0**2/3)
dsigunpvalue=1.0
drunptemp=1.0
    

i=0
dv=dv0
if M0>0:
    du0=M0/N
elif M0==0:
    du0=1/N

du=du0

if bdytype=="edd" or bdytype=="fulledd":
    bdyvalue=1.0
    dsigunp[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
else:
    bdyvalue=0.0
    dsigunp[0]=0.0
 
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

drunp=np.append(drunp,np.nan)
dsigunp=np.append(dsigunp,np.nan)
dphinpuf=np.append(dphinpuf,np.nan)

TempTol=1 
TempTolv=1
Eres=False
while max(urange)<umax and i<50000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    #drunp=np.append(drunp,np.nan)
    #dsigunp=np.append(dsigunp,np.nan)
    #dphinpuf=np.append(dphinpuf,np.nan)
    
    du=du0*Tol*TempTol/max([abs(dsigunpvalue),abs(drunptemp)])
    #du=abs(du0*Tol*TempTol/mth.exp(np.nanmax(signpf[0,:])))
    #dui=abs(mth.exp(np.nanmax(signpf[0,:]))/(du0*Tol))
    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(1*Nul)
            #print(rplus)
        else:
            du=1/Nul
    
    
    
    
    print("%.24f" % urange[i])
    print("%.24f" % du)
    
    if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        u=urange[i]+du
        phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    else:
        phinpf[1][0]=0.0
        dphinpuf[i+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**2.0)
    
    
    signpf[1][0]=(signpf[0][0]+du*dsigunp[i])*(1.0-bdyvalue)+np.log(abs(1.0-2.0*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]**2.0))*bdyvalue
    #dsigunp[i+1]=2.0*(M0*rnpf[1][0]-Q**2.0)/(rnpf[1][0]*(Q**2.0+rnpf[1][0]*(-2*M0+rnpf[1][0])))*drunp[i+1]*bdyvalue
    dsigunp[i+1]=2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drunp[i+1]*bdyvalue

    massnpf[1][0]=M0
    
    
    

    
    for j in range(0,Nv-1):
        answer=dnu.x4giveralt(0,j,du,dv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
        rnpf[1][j+1]=answer[0]
        phinpf[1][j+1]=answer[1]
        signpf[1][j+1]=answer[2]
        massnpf[1][j+1]=answer[3]
        #drunptemp[j+1]=answer[4]
        drunptemp=answer[4]
        dsigunpvalue=answer[5]
        exterm1=answer[6]
        exterm2=answer[7]
        exterm3=answer[8]
    
            
            
    #if any(abs((np.exp(signpf[0][:])-np.exp(signpf[1][:]))/np.exp(signpf[0][:]))>0.01):  
    if float(dsigunpvalue*du)>0.1:
        TempTol=TempTol/2
        Eres=True
    else:
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
    
        drunp=np.append(drunp,np.nan)
        dsigunp=np.append(dsigunp,np.nan)
        dphinpuf=np.append(dphinpuf,np.nan)
    
        rnpf[0,:]=rnpf[1,:]
        phinpf[0,:]=phinpf[1,:]
        signpf[0,:]=signpf[1,:]
        massnpf[0,:]=massnpf[1,:]
    
        urange=np.append(urange,urange[i]+du)
        
        if Eres==True:
            TempTol=2*TempTol
        else:
            TempTol=1
        Eres=False
        i+=1
    
    print(TempTol)
    print(exterm1)
    print(exterm2)
    print(exterm3)
    print('---')
    
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    #print(urange)
     
    #print(urange)
  
    
    #du=abs(du0*Tol/np.nanmin(drunptemp))

    
    
    
    
    if rnpf[1][0]<0.0 or np.isnan(du):
        break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


0.000000000000000000000000
0.001000000000000000020817
1
-1.560223803653126
0.15041643041241004
0.4530386122135039
---
0.001000000000000000020817
0.001000000000000000020817
1
-1.5609310450971214
0.15041595780707942
0.45334587798054293
---
0.002000000000000000041633
0.001000000000000000020817
1
-1.561639087563443
0.15041548494189083
0.4536534869827228
---
0.003000000000000000062450
0.001000000000000000020817
1
-1.562347931776912
0.1504150118166649
0.45396143981641396
---
0.004000000000000000083267
0.001000000000000000020817
1
-1.5630575792367696
0.15041453843140182
0.4542697370567585
---
0.005000000000000000104083
0.001000000000000000020817
1
-1.5637680316340854
0.1504140647852139
0.45457837921944616
---
0.006000000000000000124900
0.001000000000000000020817
1
-1.564479289970768
0.15041359087827771
0.4548873668885056
---
0.007000000000000000145717
0.001000000000000000020817
1
-1.5651913557815829
0.1504131167102382
0.4551967006202198
---
0.008000000000000000166533
0.00100000000000000002081

1
-1.611740290096849
0.15038270340678073
0.4754076937318538
---
0.071000000000000049182880
0.001000000000000000020817
1
-1.6125069741863036
0.15038221184290457
0.47574039547641084
---
0.072000000000000050071058
0.001000000000000000020817
1
-1.6132745600503466
0.15038171999623406
0.47607348233718455
---
0.073000000000000050959237
0.001000000000000000020817
1
-1.614043049081671
0.1503812278664132
0.4764069550063747
---
0.074000000000000051847415
0.001000000000000000020817
1
-1.6148124431028075
0.15038073545299824
0.4767408140857916
---
0.075000000000000052735594
0.001000000000000000020817
1
-1.6155827434886874
0.15038024275581063
0.4770750602520468
---
0.076000000000000053623772
0.001000000000000000020817
1
-1.61635395211868
0.15037974977422922
0.47740969413676004
---
0.077000000000000054511951
0.001000000000000000020817
1
-1.6171260703499566
0.15037925650807538
0.4777447164088129
---
0.078000000000000055400129
0.001000000000000000020817
1
-1.6178990994046962
0.15037876295717037
0.478080

1
-1.6676748523692293
0.15034759102978018
0.4996636520338202
---
0.141000000000000097477582
0.001000000000000000020817
1
-1.668508638213805
0.15034707878094275
0.5000249561950693
---
0.142000000000000098365760
0.001000000000000000020817
1
-1.6693434446544368
0.1503465662229969
0.5003866944545132
---
0.143000000000000099253938
0.001000000000000000020817
1
-1.6701792735243406
0.1503460533554982
0.5007488675271495
---
0.144000000000000100142117
0.001000000000000000020817
1
-1.6710161259853336
0.15034554017817817
0.5011114761836327
---
0.145000000000000101030295
0.001000000000000000020817
1
-1.6718540048049078
0.15034502669041686
0.5014745212065588
---
0.146000000000000101918474
0.001000000000000000020817
1
-1.6726929113971003
0.15034451289194642
0.5018380033445304
---
0.147000000000000102806652
0.001000000000000000020817
1
-1.6735328470089867
0.1503439987824986
0.5022019233887759
---
0.148000000000000103694830
0.001000000000000000020817
1
-1.6743738146105465
0.15034348436136505
0.50256628

1
-1.7286089754242466
0.15031096438896702
0.5260463949683936
---
0.211000000000000159650071
0.001000000000000000020817
1
-1.7295188836232531
0.1503104294889206
0.526440015768452
---
0.212000000000000160538249
0.001000000000000000020817
1
-1.7304299520092044
0.15030989424962618
0.5268341282426355
---
0.213000000000000161426428
0.001000000000000000020817
1
-1.7313421832714748
0.150309358670818
0.5272287332843684
---
0.214000000000000162314606
0.001000000000000000020817
1
-1.732255579531059
0.1503088227516961
0.5276238317789561
---
0.215000000000000163202785
0.001000000000000000020817
1
-1.7331701427739612
0.15030828649217076
0.5280194246461353
---
0.216000000000000164090963
0.001000000000000000020817
1
-1.734085875202882
0.1503077498917086
0.5284155127798923
---
0.217000000000000164979141
0.001000000000000000020817
1
-1.7350027792336653
0.15030721294977673
0.5288120970729088
---
0.218000000000000165867320
0.001000000000000000020817
1
-1.7359208567812694
0.1503066756656635
0.5292091784666

1
-1.7962264996412889
0.15027211647446184
0.555268618493629
---
0.282000000000000194955163
0.001000000000000000020817
1
-1.7972245518106922
0.1502715562973892
0.5556995024636191
---
0.283000000000000195843342
0.001000000000000000020817
1
-1.7982239341408799
0.15027099574604036
0.5561309474172577
---
0.284000000000000196731520
0.001000000000000000020817
1
-1.7992246486853956
0.15027043481996927
0.5565629544594247
---
0.285000000000000197619698
0.001000000000000000020817
1
-1.8002266987837432
0.15026987351864426
0.5569955246367414
---
0.286000000000000198507877
0.001000000000000000020817
1
-1.8012300861342305
0.15026931184161846
0.5574286590394884
---
0.287000000000000199396055
0.001000000000000000020817
1
-1.8022348139982105
0.15026874978827112
0.5578623587296355
---
0.288000000000000200284234
0.001000000000000000020817
1
-1.8032408849407724
0.15026818735806852
0.5582966247702215
---
0.289000000000000201172412
0.001000000000000000020817
1
-1.804248301605157
0.1502676245504773
0.55873145

...

1
-0.016789205107440582
1.7713079771700688e-07
-5.8507300411001495
---
4.981639809728441115055375
0.001000000000000000020817
1
-0.01691133282774133
1.7713514902489513e-07
-5.860256755635618
---
4.982639809728441449010461
0.001000000000000000020817
1
-0.017034217714034638
1.7713953651526795e-07
-5.869825394341888
---
4.983639809728441782965547
0.001000000000000000020817
1
-0.017157863613132242
1.77143951861881e-07
-5.879436239997204
---
4.984639809728442116920633
0.001000000000000000020817
1
-0.017282274428245207
1.771484033920499e-07
-5.88908957324108
---
4.985639809728442450875718
0.001000000000000000020817
1
-0.017407454014289905
1.771528827795306e-07
-5.898785678898122
---
4.986639809728442784830804
0.001000000000000000020817
1
-0.017533406273034104
1.7715740112723399e-07
-5.908524844926433
---
4.987639809728443118785890
0.001000000000000000020817
1
-0.0176601351805195
1.771619501089317e-07
-5.918307360438746
---
4.988639809728443452740976
0.001000000000000000020817
1
-0.01778764457

1
-0.027026106677540802
1.7751051046829072e-07
-6.610282876874152
---
5.049639809728463824001210
0.001000000000000000020817
1
-0.027209396548636
1.7751754836558298e-07
-6.623438664071557
---
5.050639809728464157956296
0.001000000000000000020817
1
-0.02739374787509351
1.7752463359079925e-07
-6.636662205941946
---
5.051639809728464491911382
0.001000000000000000020817
1
-0.02757916587429538
1.775317661446945e-07
-6.649954011162862
---
5.052639809728464825866467
0.001000000000000000020817
1
-0.027765655769729888
1.7753894602802536e-07
-6.6633145939469784
---
5.053639809728465159821553
0.001000000000000000020817
1
-0.027953222851054598
1.775461760171722e-07
-6.676744474482632
---
5.054639809728465493776639
0.001000000000000000020817
1
-0.028141872388831243
1.7755346166412995e-07
-6.690244176031257
---
5.055639809728465827731725
0.001000000000000000020817
1
-0.028331609697031288
1.7756079464282795e-07
-6.703814228860239
---
5.056639809728466161686811
0.001000000000000000020817
1
-0.028522440

1
-0.04219167579756001
1.7811481163293222e-07
-7.689304469533198
---
5.117639809728486532947045
0.001000000000000000020817
1
-0.04246045449031809
1.7812587194981335e-07
-7.708520830611576
---
5.118639809728486866902131
0.001000000000000000020817
1
-0.04273071784823647
1.781370046384667e-07
-7.727853930010298
---
5.119639809728487200857217
0.001000000000000000020817
1
-0.04300247337864319
1.781482069244013e-07
-7.7473047737903755
---
5.120639809728487534812302
0.001000000000000000020817
1
-0.0432757285759877
1.7815948436006127e-07
-7.766874380038757
---
5.121639809728487868767388
0.001000000000000000020817
1
-0.04355049107416348
1.7817083694662282e-07
-7.786563777770539
---
5.122639809728488202722474
0.001000000000000000020817
1
-0.04382676846765108
1.7818226190961213e-07
-7.806374007447658
---
5.123639809728488536677560
0.001000000000000000020817
1
-0.04410456847905078
1.7819376202586664e-07
-7.8263061186483185
---
5.124639809728488870632646
0.001000000000000000020817
1
-0.044383898791

1
-0.06426902013145411
1.790534610376712e-07
-9.323197599094613
---
5.185639809728509241892880
0.001000000000000000020817
1
-0.06465856958713422
1.7907048662243016e-07
-9.353347576631421
---
5.186639809728509575847966
0.001000000000000000020817
1
-0.06505024702048484
1.7908761798927557e-07
-9.383717775477862
---
5.187639809728509909803051
0.001000000000000000020817
1
-0.0654440645440391
1.7910485514018797e-07
-9.414310458659259
---
5.188639809728510243758137
0.001000000000000000020817
1
-0.06584003431207451
1.7912219807715778e-07
-9.445127921761554
---
5.189639809728510577713223
0.001000000000000000020817
1
-0.0662381686612782
1.7913964957789694e-07
-9.476172489641499
---
5.190639809728510911668309
0.001000000000000000020817
1
-0.06663847994698906
1.7915721242012765e-07
-9.507446518220775
---
5.191639809728511245623395
0.001000000000000000020817
1
-0.06704098069574237
1.7917488383021285e-07
-9.53895239562641
---
5.192639809728511579578480
0.001000000000000000020817
1
-0.067445683510234

1
-0.09631014361080929
1.8048850669742197e-07
-12.019683683682558
---
5.253639809728531950838715
0.001000000000000000020817
1
-0.09687832677401653
1.8051442430836737e-07
-12.072023460198977
---
5.254639809728532284793800
0.001000000000000000020817
1
-0.09744977887954255
1.8054050340795465e-07
-12.124842784828616
---
5.255639809728532618748886
0.001000000000000000020817
1
-0.09802452348387729
1.8056674677598945e-07
-12.178147904489123
---
5.256639809728532952703972
0.001000000000000000020817
1
-0.09860258442961514
1.8059314886504856e-07
-12.231945177347825
---
5.257639809728533286659058
0.001000000000000000020817
1
-0.0991839858301339
1.8061972078232077e-07
-12.286241065478265
---
5.258639809728533620614144
0.001000000000000000020817
1
-0.09976875213976084
1.806464542047269e-07
-12.341042145472159
---
5.259639809728533954569230
0.001000000000000000020817
1
-0.10035690802243337
1.8067335191227216e-07
-12.396355109792978
---
5.260639809728534288524315
0.001000000000000000020817
1
-0.10094

In [7]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [8]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

6962


In [9]:
#print(rnpf[1,0])

In [10]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [11]:
#print(rnpf)

In [12]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [13]:
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD]))
np.save('urange',urange)
np.save('vrange',vrange)

In [14]:
stop = timeit.default_timer()

In [15]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

846.8773182430014 seconds
14.114621970716689 minutes
0.23524369951194482 hours
34810000 points
41103.946522289174 points per second
Took 552.12 less minutes


conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            